# Lunar Hazard Detection System - Demo

This notebook demonstrates the complete lunar hazard detection pipeline using YOLOv8.

## Overview
- Data acquisition and preparation
- Model training and evaluation
- Hazard detection on lunar surfaces
- Performance analysis

## 1. Setup and Installation

In [ ]:
# Install required packages
# !pip install -r ../requirements.txt

# Import necessary libraries
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import os

# Add project root to path
project_root = Path('../').resolve()
sys.path.append(str(project_root))

# Import project modules
from scripts.augment_lunar_data import LunarAugmentation
from ultralytics import YOLO

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Project root: {project_root}")

## 2. Data Preparation and Augmentation

In [ ]:
# Initialize data augmentation
augmentor = LunarAugmentation(input_dir='../data/raw', output_dir='../data/augmented')

# Create sample lunar surface
def create_sample_lunar_surface(width=640, height=480):
    """Create a sample lunar surface for demonstration"""
    # Base lunar surface (gray)
    surface = np.full((height, width, 3), 128, dtype=np.uint8)
    
    # Add lunar regolith texture
    noise = np.random.normal(0, 15, (height, width, 3))
    surface = np.clip(surface + noise.astype(np.int16), 0, 255).astype(np.uint8)
    
    return surface

# Generate sample image
sample_surface = create_sample_lunar_surface()
plt.figure(figsize=(10, 6))
plt.imshow(sample_surface)
plt.title('Sample Lunar Surface')
plt.axis('off')
plt.show()

# Save sample image
sample_path = Path('../data/raw/sample_lunar.jpg')
sample_path.parent.mkdir(parents=True, exist_ok=True)
cv2.imwrite(str(sample_path), cv2.cvtColor(sample_surface, cv2.COLOR_RGB2BGR))
print(f"Sample image saved to: {sample_path}")

In [ ]:
# Apply lunar-specific augmentations
augmented_files = augmentor.augment_image(sample_path, 'low_light')
print(f"Generated {len(augmented_files)} augmented images")

# Display original vs augmented
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Original
original = cv2.imread(str(sample_path))
original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
axes[0].imshow(original)
axes[0].set_title('Original Lunar Surface')
axes[0].axis('off')

# Augmented
if augmented_files:
    augmented = cv2.imread(str(augmented_files[0]))
    augmented = cv2.cvtColor(augmented, cv2.COLOR_BGR2RGB)
    axes[1].imshow(augmented)
    axes[1].set_title('Low-Light Augmented')
    axes[1].axis('off')

plt.tight_layout()
plt.show()

## 3. Model Loading and Setup

In [ ]:
# Load pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # Using nano model for demo

# Display model information
print("Model Information:")
print(f"Model type: {type(model)}")
print(f"Model path: {model.ckpt_path}")

# Test model on sample image
results = model(sample_path)
print(f"\nInference completed. Found {len(results[0].boxes)} objects.")

# Display results
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(original)
ax.set_title('YOLOv8 Detection Results (Pre-trained)')
ax.axis('off')

# Draw bounding boxes if any detections
if len(results[0].boxes) > 0:
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = box.conf[0].cpu().numpy()
        cls = int(box.cls[0].cpu().numpy())
        
        # Draw rectangle
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                           fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        
        # Add label
        ax.text(x1, y1-10, f'Class {cls}: {conf:.2f}', 
               color='red', fontsize=12, weight='bold')

plt.show()

## 4. Lunar Hazard Detection Classes

Our system is designed to detect the following lunar hazards:

1. **Crater**: Impact craters of various sizes
2. **Rock**: Surface rocks and boulders  
3. **Shadow Region**: Permanently shadowed areas
4. **Dust Devil**: Dust movement patterns
5. **Slope**: Steep terrain features
6. **Lunar Module**: Human-made objects

In [ ]:
# Create synthetic lunar hazards for demonstration
def create_lunar_hazards_demo():
    """Create a synthetic lunar surface with various hazards"""
    # Create base surface
    surface = create_sample_lunar_surface(640, 480)
    
    # Add synthetic crater
    center_x, center_y = 320, 240
    radius = 50
    y, x = np.ogrid[:480, :640]
    crater_mask = np.sqrt((x - center_x)**2 + (y - center_y)**2) <= radius
    surface[crater_mask] = surface[crater_mask] * 0.5  # Darken crater
    
    # Add synthetic rocks
    rock_positions = [(100, 100), (500, 150), (200, 350), (450, 400)]
    for pos in rock_positions:
        rock_mask = ((x - pos[0])**2 + (y - pos[1])**2) <= 25
        surface[rock_mask] = np.clip(surface[rock_mask] * 1.3 + 30, 0, 255)
    
    # Add shadow region
    shadow_region = slice(350, 480), slice(100, 300)
    surface[shadow_region] = surface[shadow_region] * 0.3
    
    return surface, rock_positions

# Generate demo image with hazards
demo_surface, rock_pos = create_lunar_hazards_demo()

# Save demo image
demo_path = Path('../data/raw/lunar_hazards_demo.jpg')
cv2.imwrite(str(demo_path), cv2.cvtColor(demo_surface, cv2.COLOR_RGB2BGR))

# Display with annotations
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(demo_surface)
ax.set_title('Synthetic Lunar Surface with Hazards')
ax.axis('off')

# Add manual annotations for demonstration
ax.add_patch(plt.Circle((320, 240), 50, fill=False, edgecolor='red', linewidth=3, label='Crater'))
ax.text(320, 240, 'CRATER', color='red', fontsize=12, weight='bold', ha='center')

for i, pos in enumerate(rock_pos):
    ax.add_patch(plt.Circle(pos, 5, fill=False, edgecolor='green', linewidth=2))
    ax.text(pos[0], pos[1], f'ROCK {i+1}', color='green', fontsize=10, weight='bold')

ax.add_patch(plt.Rectangle((100, 350), 200, 130, fill=False, edgecolor='blue', linewidth=3, label='Shadow Region'))
ax.text(200, 420, 'SHADOW REGION', color='blue', fontsize=12, weight='bold')

ax.legend()
plt.show()

print(f"Demo image saved to: {demo_path}")

## 5. Model Training Demo

For demonstration purposes, we'll show how the training script would be used. In a real scenario, you would have a proper lunar dataset with annotations.

In [ ]:
# Show training configuration
training_config = {
    'model': 'yolov8n.pt',
    'epochs': 100,
    'batch_size': 16,
    'imgsz': 640,
    'optimizer': 'AdamW',
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'patience': 20,
    'save': True,
    'save_period': 10,
    'val': True,
    'plots': True
}

print("Lunar-Optimized Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

# Lunar-specific optimizations
lunar_optimizations = {
    'mosaic': 0.5,          # Reduced mosaic augmentation
    'mixup': 0.1,           # Reduced mixup for clearer features
    'copy_paste': 0.0,      # Disabled for lunar surfaces
    'degrees': 10,          # Reduced rotation
    'translate': 0.1,       # Reduced translation
    'scale': 0.3,           # Reduced scaling
    'shear': 2,             # Reduced shear
    'perspective': 0.0,     # Disabled perspective transform
    'flipud': 0.0,          # Disabled vertical flip
    'fliplr': 0.5,          # Keep horizontal flip
    'hsv_h': 0.01,          # Reduced HSV hue augmentation
    'hsv_s': 0.3,           # Reduced saturation augmentation
    'hsv_v': 0.2,           # Reduced value augmentation
    'auto_augment': 'randaugment'
}

print("\nLunar-Specific Optimizations:")
for key, value in lunar_optimizations.items():
    print(f"  {key}: {value}")

## 6. Performance Evaluation

Let's evaluate the pre-trained model on our synthetic lunar surface and analyze the results.

In [ ]:
# Test model on synthetic lunar surface
results = model(demo_path)

print("Detection Results:")
print(f"Number of detections: {len(results[0].boxes)}")

if len(results[0].boxes) > 0:
    print("\nDetailed Results:")
    for i, box in enumerate(results[0].boxes):
        cls = int(box.cls[0].cpu().numpy())
        conf = float(box.conf[0].cpu().numpy())
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        print(f"  Detection {i+1}: Class {cls}, Confidence: {conf:.3f}, Box: ({x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f})")

# Visualize results
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(demo_surface)
ax.set_title('Model Detection Results on Synthetic Lunar Surface')
ax.axis('off')

# Draw detected bounding boxes
if len(results[0].boxes) > 0:
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        
        # Draw rectangle
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                           fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        
        # Add label
        ax.text(x1, y1-10, f'Class {cls}: {conf:.2f}', 
               color='red', fontsize=12, weight='bold')

plt.show()

## 7. Scenario Simulation

Let's demonstrate the scenario simulation capabilities for generating test data.

In [ ]:
# Import simulation module
from simulation.test_lunar_scenarios import LunarScenarioSimulator

# Initialize simulator
simulator = LunarScenarioSimulator(output_dir='../simulation/scenarios')

# Generate a small crater field scenario for demo
print("Generating crater field scenario...")
scenarios = simulator.generate_scenario('crater_field_demo', num_images=3)

# Display generated scenarios
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Generated Lunar Scenarios - Crater Field Demo')

for i, scenario in enumerate(scenarios):
    image_path = Path(scenario['filename'])
    if image_path.exists():
        img = cv2.imread(str(image_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i].imshow(img)
        axes[i].set_title(f'Scenario {i+1}\nHazards: {len(scenario["hazards"])}')
        axes[i].axis('off')

plt.tight_layout()
plt.show()

print(f"Generated {len(scenarios)} scenarios in: ../simulation/scenarios/")

## 8. Performance Analysis

Let's analyze the model's performance characteristics and create some performance metrics.

In [ ]:
# Test model performance on multiple images
test_images = [
    sample_path,
    demo_path,
    demo_path  # Test same image multiple times for consistency
]

performance_results = []

for i, img_path in enumerate(test_images):
    # Time the inference
    import time
    start_time = time.time()
    results = model(img_path)
    inference_time = time.time() - start_time
    
    num_detections = len(results[0].boxes) if results[0].boxes is not None else 0
    avg_confidence = np.mean([float(box.conf) for box in results[0].boxes]) if num_detections > 0 else 0
    
    performance_results.append({
        'image': f'image_{i+1}',
        'inference_time': inference_time,
        'num_detections': num_detections,
        'avg_confidence': avg_confidence
    })

# Display performance results
import pandas as pd
df = pd.DataFrame(performance_results)
print("Performance Analysis:")
print(df)

# Create performance visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Inference time plot
ax1.bar(df['image'], df['inference_time'], color='skyblue', alpha=0.7)
ax1.set_title('Inference Time per Image')
ax1.set_ylabel('Time (seconds)')
ax1.set_xlabel('Test Image')
for i, v in enumerate(df['inference_time']):
    ax1.text(i, v + 0.001, f'{v:.3f}s', ha='center', va='bottom')

# Detections plot
ax2.bar(df['image'], df['num_detections'], color='lightgreen', alpha=0.7)
ax2.set_title('Number of Detections per Image')
ax2.set_ylabel('Number of Detections')
ax2.set_xlabel('Test Image')
for i, v in enumerate(df['num_detections']):
    ax2.text(i, v + 0.1, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Calculate summary statistics
print("\nSummary Statistics:")
print(f"Average inference time: {df['inference_time'].mean():.3f}s ± {df['inference_time'].std():.3f}s")
print(f"Average detections per image: {df['num_detections'].mean():.1f} ± {df['num_detections'].std():.1f}")
print(f"Average confidence: {df['avg_confidence'].mean():.3f} ± {df['avg_confidence'].std():.3f}")

## 9. Next Steps and Recommendations

This demo shows the basic capabilities of the lunar hazard detection system. Here are the recommended next steps:

In [ ]:
# Display next steps
next_steps = """
### Next Steps for Lunar Hazard Detection:

1. **Data Collection**
   - Download real lunar images from NASA LRO
   - Collect images from USGS Astrogeology
   - Create custom dataset with proper annotations

2. **Model Training**
   - Train YOLOv8 on lunar dataset
   - Fine-tune for specific hazard types
   - Optimize for low-light conditions

3. **Advanced Augmentation**
   - Implement PSR (Permanently Shadowed Region) simulation
   - Add realistic lunar lighting conditions
   - Create domain-specific augmentations

4. **Performance Optimization**
   - Test different YOLOv8 model sizes
   - Optimize for real-time processing
   - Implement hardware acceleration

5. **Evaluation and Testing**
   - Generate comprehensive test scenarios
   - Evaluate on real lunar images
   - Benchmark against other models

6. **Deployment**
   - Create web interface for model
   - Deploy as API service
   - Integrate with lunar rover systems
"""

print(next_steps)

# Show file structure
print("\nProject Structure:")
for root, dirs, files in os.walk(project_root):
    level = root.replace(str(project_root), '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... and {len(files) - 5} more files')

## 10. Conclusion

This notebook demonstrated the complete lunar hazard detection pipeline:

- ✅ Data preparation and augmentation
- ✅ Model loading and inference
- ✅ Synthetic hazard generation
- ✅ Performance analysis
- ✅ Scenario simulation

The system is ready for real lunar data and can be extended with:
- Real lunar image datasets
- Custom-trained models
- Advanced evaluation metrics
- Deployment capabilities

### Key Takeaways:
1. YOLOv8 provides excellent baseline performance
2. Lunar-specific augmentations are crucial for performance
3. Synthetic data generation helps with testing and validation
4. The modular architecture allows easy extension and customization

For production use, train the model on real lunar datasets and fine-tune for specific mission requirements.